In [46]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import PunktSentenceTokenizer
from nltk.stem import SnowballStemmer, LancasterStemmer
import matplotlib.pyplot as plt
import pickle

In [47]:
df = pd.read_csv("dataset2.csv")
df.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,descriptions,relevant_text,latitude
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,...,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"Evaluated as espresso. Sweet-toned, deeply ric...","A.R.C.\n“Sweety” Espresso Blend\nHong Kong, Ch...",9.145000
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,...,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","Evaluated as espresso. Sweetly tart, floral-to...","A.R.C.\nFlora Blend Espresso\nHong Kong, China...",-8.783195
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,...,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","Crisply sweet, cocoa-toned. Lemon blossom, roa...",Revel Coffee\nEthiopia Shakiso Mormora\nBillin...,6.515691
3,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRoast House\nEthiopia Suke Quto...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,...,9.0,9.0,8.0,NaN,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,"Delicate, sweetly spice-toned. Pink peppercorn...","Roast House\nEthiopia Suke Quto\nSpokane, Wash...",9.066516
4,https://www.coffeereview.com/review/ethiopia-g...,\n\n\n94\n\n\nBig Creek Coffee Roasters\nEthi...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,...,9.0,9.0,8.0,NaN,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,"Deeply sweet, subtly pungent. Honey, pear, tan...",Big Creek Coffee Roasters\nEthiopia Gedeb Halo...,5.917024


In [48]:
count = 0
for i in df["desc_1"]:
    if "three parts milk" in i:
        #print(i)
        count += 1
print(count)

210


In [49]:
def process_content(description):
    l = []
    try:
        words = nltk.word_tokenize(description.lower().replace("-"," "))
        #print(words)
        tagged = nltk.pos_tag(words)
        #print(tagged)

        # All chunks
        chunk_gram = r"""Chunk: {<NNP>?<RB.?>*<VBD>*<JJ.?>*<NNP.?>*<NN.?>*<NN.?>*}"""
        chunk_parser = nltk.RegexpParser(chunk_gram)
        chunked = chunk_parser.parse(tagged)

        # Finding the finish description
        finish_gram = r"""Chunk: {<NNP>?<RB.?>*<VBD>*<VBN>*<JJ.?>*<NNP.?>*<NN.?>*<NN.?>*<IN>+<DT>+<JJ.?>*<NN>*}"""
        finish_parser = nltk.RegexpParser(finish_gram)
        finish_chunks = finish_parser.parse(tagged)

        # Finding the milk description
        
        # print(description)

        for chunk in chunked:
            if hasattr(chunk,"label"):
                if chunk.label() == "Chunk":
                    l.append(chunk)
            # print(chunk)
        #print("---Finish---")
        # if len(finish_chunks) == 0:
        #     for chunk in chunked:
        #         if hasattr(chunk,"label"):
        #             if chunk.label() == "Chunk":
        #                 for j in chunk:
        #                     print(j[0])
        # for chunk in finish_chunks:
        #     if hasattr(chunk,"label"):
        #         if chunk.label() == "Chunk":
        #             print(chunk)
        #             # for j in chunk:
        #             #     print(j[0])
        #print("---Finish done---")

    except Exception as e:
        print(str(e))

    return l

chunks = process_content(df["desc_1"][0])
chunks

[Tree('Chunk', [('espresso', 'NN')]),
 Tree('Chunk', [('sweet', 'NN')]),
 Tree('Chunk', [('toned', 'VBD')]),
 Tree('Chunk', [('deeply', 'RB'), ('rich', 'JJ')]),
 Tree('Chunk', [('chocolaty', 'NN')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('dark', 'JJ'), ('chocolate', 'NN')]),
 Tree('Chunk', [('narcissus', 'NN')]),
 Tree('Chunk', [('grapefruit', 'NN'), ('zest', 'NN')]),
 Tree('Chunk', [('black', 'JJ'), ('cherry', 'NN')]),
 Tree('Chunk', [('aroma', 'NN')]),
 Tree('Chunk', [('cup', 'NN')]),
 Tree('Chunk', [('plush', 'NN')]),
 Tree('Chunk', [('syrupy', 'JJ'), ('mouthfeel', 'NN')]),
 Tree('Chunk', [('resonant', 'NN')]),
 Tree('Chunk', [('flavor', 'NN')]),
 Tree('Chunk', [('saturated', 'VBD'), ('finish', 'JJ')]),
 Tree('Chunk', [('parts', 'NNS'), ('milk', 'NN')]),
 Tree('Chunk', [('rich', 'JJ'), ('chocolate', 'NN'), ('tones', 'NNS')]),
 Tree('Chunk', [('intimations', 'NNS')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('black', 'JJ'

In [50]:
useless = ["milk","short","long","cup","flavor",
    "flavour","www","http","visit","information"
    "cafe","coffee","coffees","blend","shot"]

characteristic_names = ["finish","tone","tones","toned","mouthfeel","paste"
    "note","notes"]

def clean_chunk(chunk):
    grammar = [i[1] for i in chunk]
    words = [i[0] for i in chunk]
    for i in characteristic_names:
        for j in chunk:
            if j[0] == i:
                chunk.remove(j)
    for i in useless:
        if i in "".join(words):
            return None
    return chunk

In [51]:
clean = [clean_chunk(i) for i in chunks if clean_chunk(i) is not None]
clean

[Tree('Chunk', [('espresso', 'NN')]),
 Tree('Chunk', [('sweet', 'NN')]),
 Tree('Chunk', []),
 Tree('Chunk', [('deeply', 'RB'), ('rich', 'JJ')]),
 Tree('Chunk', [('chocolaty', 'NN')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('dark', 'JJ'), ('chocolate', 'NN')]),
 Tree('Chunk', [('narcissus', 'NN')]),
 Tree('Chunk', [('grapefruit', 'NN'), ('zest', 'NN')]),
 Tree('Chunk', [('black', 'JJ'), ('cherry', 'NN')]),
 Tree('Chunk', [('aroma', 'NN')]),
 Tree('Chunk', [('plush', 'NN')]),
 Tree('Chunk', [('syrupy', 'JJ')]),
 Tree('Chunk', [('resonant', 'NN')]),
 Tree('Chunk', [('saturated', 'VBD')]),
 Tree('Chunk', [('rich', 'JJ'), ('chocolate', 'NN')]),
 Tree('Chunk', [('intimations', 'NNS')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('black', 'JJ'), ('cherry', 'NN')]),
 Tree('Chunk', [('floral', 'JJ')]),
 Tree('Chunk', [('citrus', 'NN'), ('zest', 'NN')])]

In [52]:
stemmer = PorterStemmer()
stemmer = SnowballStemmer(language = "english")

In [53]:
def extract_characteristics(raw):
    characteristics = []
    for chunk in raw:
        grammar = [i[1] for i in chunk]
        words = [i[0] for i in chunk]
        clean = clean_chunk(chunk)
        if clean is None:
            continue

        if len(clean) == 0:
            continue
        
        if len(clean) == 1:
            if grammar[0] != "VBD":
                characteristics.append(stemmer.stem(words[0]))
            continue

        if all(i=="NN" for i in grammar):
            characteristics.append(stemmer.stem(words[0]))
            continue
        
        elif grammar[0].startswith('JJ') and grammar[1].startswith('NN'):
            characteristics.append(stemmer.stem(words[1]))
            continue

        elif grammar[0].startswith('RB') and grammar[1].startswith('JJ'):
            characteristics.append(stemmer.stem(words[1]))
            continue
    return np.array(characteristics)

extract_characteristics(chunks)

array(['espresso', 'sweet', 'rich', 'chocolati', 'vanilla', 'chocol',
       'narcissus', 'grapefruit', 'cherri', 'aroma', 'plush', 'syrupi',
       'reson', 'chocol', 'intim', 'vanilla', 'cherri', 'floral',
       'citrus'], dtype='<U10')

In [54]:
df["characteristics"] = df["desc_1"].apply(lambda x:extract_characteristics(process_content(x)))

In [55]:
df.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,descriptions,relevant_text,latitude,characteristics
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,...,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"Evaluated as espresso. Sweet-toned, deeply ric...","A.R.C.\n“Sweety” Espresso Blend\nHong Kong, Ch...",9.145000,"[espresso, sweet, rich, chocolati, vanilla, ch..."
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,...,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","Evaluated as espresso. Sweetly tart, floral-to...","A.R.C.\nFlora Blend Espresso\nHong Kong, China...",-8.783195,"[espresso, floral, honeysuckl, oak, chocol, th..."
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,...,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","Crisply sweet, cocoa-toned. Lemon blossom, roa...",Revel Coffee\nEthiopia Shakiso Mormora\nBillin...,6.515691,"[crispli, cocoa, blossom, date, rice, pepperco..."
3,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRoast House\nEthiopia Suke Quto...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,...,9.0,8.0,NaN,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,"Delicate, sweetly spice-toned. Pink peppercorn...","Roast House\nEthiopia Suke Quto\nSpokane, Wash...",9.066516,"[delic, peppercorn, date, myrrh, lavend, aroma..."
4,https://www.coffeereview.com/review/ethiopia-g...,\n\n\n94\n\n\nBig Creek Coffee Roasters\nEthi...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,...,9.0,8.0,NaN,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,"Deeply sweet, subtly pungent. Honey, pear, tan...",Big Creek Coffee Roasters\nEthiopia Gedeb Halo...,5.917024,"[sweet, pungent, honey, pear, tangerin, zest, ..."


In [56]:
all_characteristics = set([])
for i in df["characteristics"]:
    all_characteristics.update(i)


In [57]:
df["coffee"] = df["name"] + "-" + df["roaster"]
df.head(3)

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,aftertaste,with_milk,desc_1,desc_2,desc_3,descriptions,relevant_text,latitude,characteristics,coffee
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,...,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"Evaluated as espresso. Sweet-toned, deeply ric...","A.R.C.\n“Sweety” Espresso Blend\nHong Kong, Ch...",9.145000,"[espresso, sweet, rich, chocolati, vanilla, ch...",“Sweety” Espresso Blend-A.R.C.
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,...,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","Evaluated as espresso. Sweetly tart, floral-to...","A.R.C.\nFlora Blend Espresso\nHong Kong, China...",-8.783195,"[espresso, floral, honeysuckl, oak, chocol, th...",Flora Blend Espresso-A.R.C.
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,...,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","Crisply sweet, cocoa-toned. Lemon blossom, roa...",Revel Coffee\nEthiopia Shakiso Mormora\nBillin...,6.515691,"[crispli, cocoa, blossom, date, rice, pepperco...",Ethiopia Shakiso Mormora-Revel Coffee


In [58]:
# We want to remove characteristics that appear in less that 10% of the coffees
d = {}
for i in df["characteristics"]:
    for j in i:
        if j in d:
            d[j] += 1
        else:
            d[j] = 1

count_dict = {i:j for i,j in d.items() if j > len(df)/10}
count_dict

{'espresso': 315,
 'sweet': 1195,
 'rich': 415,
 'chocolati': 398,
 'chocol': 1558,
 'narcissus': 256,
 'cherri': 238,
 'aroma': 2255,
 'plush': 309,
 'syrupi': 536,
 'reson': 279,
 'floral': 487,
 'oak': 235,
 'gentl': 373,
 'crispli': 244,
 'cocoa': 1070,
 'delic': 230,
 'finish': 511,
 'hint': 433,
 'crisp': 536,
 'acid': 1239,
 'zest': 334,
 'juici': 553,
 'structur': 691,
 'almond': 579,
 'sandalwood': 340,
 'fruit': 435,
 'high': 271,
 'apricot': 267,
 'magnolia': 291,
 'savori': 265,
 'full': 514,
 'cedar': 761,
 'cut': 317,
 'tart': 468,
 'hazelnut': 358,
 'satini': 354,
 'flower': 351,
 'underton': 256,
 'velveti': 293,
 'bright': 376}

In [59]:
with open("keywords.p","wb") as f:
    pickle.dump(list(count_dict.keys()),f)

In [60]:
one_hot = pd.DataFrame(columns = ["coffee"] + list(count_dict.keys()))
one_hot

,coffee,espresso,sweet,rich,chocolati,chocol,narcissus,cherri,aroma,plush,...,full,cedar,cut,tart,hazelnut,satini,flower,underton,velveti,bright


In [61]:
d = {"coffee":df["coffee"]}

for i in count_dict.keys():
    d[i] = df["characteristics"].apply(lambda x: 0 if i in x else 1)

In [62]:
one_hot = pd.DataFrame(d)
one_hot.head()

,coffee,espresso,sweet,rich,chocolati,chocol,narcissus,cherri,aroma,plush,...,full,cedar,cut,tart,hazelnut,satini,flower,underton,velveti,bright
0,“Sweety” Espresso Blend-A.R.C.,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,Flora Blend Espresso-A.R.C.,0,1,1,1,0,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
2,Ethiopia Shakiso Mormora-Revel Coffee,1,1,0,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
3,Ethiopia Suke Quto-Roast House,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
4,Ethiopia Gedeb Halo Beriti-Big Creek Coffee Ro...,1,0,1,1,0,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1


In [63]:
from sklearn.neighbors import NearestNeighbors

In [64]:
one_hot[one_hot.columns[1:]]

,espresso,sweet,rich,chocolati,chocol,narcissus,cherri,aroma,plush,syrupi,...,full,cedar,cut,tart,hazelnut,satini,flower,underton,velveti,bright
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,0,1,1,1,0,1,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
2,1,1,0,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,0,1,1,0,1,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,1,1,1,1,0,1,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
2278,1,1,1,0,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,1,1
2279,1,0,1,1,1,0,0,0,1,1,...,1,1,1,1,1,1,1,1,1,1
2280,1,0,1,0,0,1,1,0,1,0,...,1,1,1,0,0,1,1,1,1,1


In [65]:
X = one_hot[one_hot.columns[1:]]
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

In [66]:
import prince

In [67]:
# We want to reduce the number of features but limit the difference in nearest neighbours
# We stop once only 10% of the sets have no overlap
l = [0]
n = 4
while True:
    n+=1
    mca = prince.MCA(n_components=n)
    mca = mca.fit(X)
    mca = mca.transform(X)
    nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(mca)
    mca_distances, mca_indices = nbrs.kneighbors(mca)
    l = []
    for i in range(len(indices)):
        l.append(len(set(indices[i][1:]) & set(mca_indices[i][1:])))
    if l.count(0) < len(l)/10:
        break
print(n)

20


In [68]:
mca = prince.MCA(n_components=n)
mca = mca.fit(X)
print(mca)
mca = mca.transform(X)
mca

MCA(n_components=20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.025074,0.318228,0.047341,0.123570,0.224134,-0.073037,-0.137452,0.100728,-0.154294,0.011988,0.082133,-0.159180,0.055034,0.100249,0.130566,0.074664,-0.056146,0.023729,0.004849,0.022251
1,-0.021282,0.241927,-0.002136,-0.087406,0.057331,0.019256,-0.087875,0.129129,-0.052701,0.005751,0.049182,-0.081455,0.054676,0.069722,-0.029042,0.076217,-0.049358,-0.044919,-0.061916,-0.007970
2,-0.023370,-0.031865,0.161014,-0.097921,-0.095761,0.092606,-0.029031,0.116825,0.046808,-0.205087,0.035979,-0.067475,-0.058231,-0.050554,0.068078,-0.110116,-0.086573,0.063906,0.039202,-0.065787
3,-0.026508,-0.088908,-0.047582,-0.105501,-0.107253,-0.014130,-0.076720,-0.052736,-0.013824,0.045572,0.082195,0.056697,-0.056453,-0.011225,0.023541,0.011821,0.008854,-0.022231,0.008662,0.001496
4,-0.012232,0.137956,0.082171,0.214607,0.128401,0.168808,-0.021614,-0.127138,-0.045692,-0.059763,-0.028747,0.000016,-0.064596,-0.023415,-0.032910,-0.095597,0.013006,-0.096601,-0.121171,-0.036891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,-0.035554,-0.028238,-0.215958,-0.107268,0.085084,0.165797,0.070559,0.010053,-0.008367,-0.034440,0.158412,-0.009948,0.138130,-0.063055,-0.109762,-0.015859,0.022111,0.058347,-0.060572,-0.046571
2278,-0.029716,-0.125359,-0.049122,-0.087392,-0.142736,0.106245,0.070983,0.044178,-0.137413,0.119197,-0.104367,-0.190307,0.044312,0.002477,-0.090236,-0.094093,-0.102015,0.004743,-0.034031,-0.089040
2279,-0.032637,-0.114409,-0.031698,0.096874,-0.044234,-0.132988,-0.083685,-0.056406,0.000127,0.088091,0.084799,0.046659,-0.049846,0.029227,0.004976,-0.001968,-0.005839,-0.054426,0.017410,0.058020
2280,-0.029880,0.020770,-0.158768,0.072235,0.176584,-0.140756,-0.062828,-0.058447,-0.044816,-0.034324,0.003087,0.063461,-0.018699,-0.059456,-0.000535,0.107118,-0.107293,0.071168,-0.080044,-0.047680


In [69]:
mca["latitude"] = 4*(df["latitude"] - df["latitude"].min()) / (df["latitude"].max()-df["latitude"].min())
mca.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,latitude
0,-0.025074,0.318228,0.047341,0.123570,0.224134,-0.073037,-0.137452,0.100728,-0.154294,0.011988,...,-0.159180,0.055034,0.100249,0.130566,0.074664,-0.056146,0.023729,0.004849,0.022251,2.022205
1,-0.021282,0.241927,-0.002136,-0.087406,0.057331,0.019256,-0.087875,0.129129,-0.052701,0.005751,...,-0.081455,0.054676,0.069722,-0.029042,0.076217,-0.049358,-0.044919,-0.061916,-0.007970,1.059924
2,-0.023370,-0.031865,0.161014,-0.097921,-0.095761,0.092606,-0.029031,0.116825,0.046808,-0.205087,...,-0.067475,-0.058231,-0.050554,0.068078,-0.110116,-0.086573,0.063906,0.039202,-0.065787,1.881079
3,-0.026508,-0.088908,-0.047582,-0.105501,-0.107253,-0.014130,-0.076720,-0.052736,-0.013824,0.045572,...,0.056697,-0.056453,-0.011225,0.023541,0.011821,0.008854,-0.022231,0.008662,0.001496,2.017993
4,-0.012232,0.137956,0.082171,0.214607,0.128401,0.168808,-0.021614,-0.127138,-0.045692,-0.059763,...,0.000016,-0.064596,-0.023415,-0.032910,-0.095597,0.013006,-0.096601,-0.121171,-0.036891,1.848946


In [70]:
# nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(mca)
# mca_distances, mca_indices = nbrs.kneighbors(mca)

In [71]:
df["recommendations"] = [i[1:] for i in indices]
df["string_recommendation"] = [[df["name"][j] for j in i[1:]] for i in indices]
for i in df["string_recommendation"][0:3]:
    print(i)

['Kenya AA Ares Phoenix Special', 'D31 The New Life', 'Indonesia Sumatra Gayo Espresso', 'Espresso Blend Classic No. 1', 'No. 7 Espresso Blend']
['Vertical Support Espresso', 'Zoom Espresso', 'Summer Night Blend Espresso', 'World 2.0 Espresso Blend', 'Organic Unicorn Espresso Blend']
['Estelí Nicaragua', 'Gigesa Ethiopia', 'Ethiopia Natural Guji Blue-Donkey 2020', 'Crescent Blend', 'Colombia Sudan Rume']


In [72]:
df.rename(columns={"desc_1":"description"},inplace=True)

In [73]:
df.head(3)

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,description,desc_2,desc_3,descriptions,relevant_text,latitude,characteristics,coffee,recommendations,string_recommendation
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,...,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"Evaluated as espresso. Sweet-toned, deeply ric...","A.R.C.\n“Sweety” Espresso Blend\nHong Kong, Ch...",9.145000,"[espresso, sweet, rich, chocolati, vanilla, ch...",“Sweety” Espresso Blend-A.R.C.,"[1127, 635, 657, 698, 205]","[Kenya AA Ares Phoenix Special, D31 The New Li..."
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,...,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","Evaluated as espresso. Sweetly tart, floral-to...","A.R.C.\nFlora Blend Espresso\nHong Kong, China...",-8.783195,"[espresso, floral, honeysuckl, oak, chocol, th...",Flora Blend Espresso-A.R.C.,"[1164, 809, 127, 2253, 710]","[Vertical Support Espresso, Zoom Espresso, Sum..."
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,...,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","Crisply sweet, cocoa-toned. Lemon blossom, roa...",Revel Coffee\nEthiopia Shakiso Mormora\nBillin...,6.515691,"[crispli, cocoa, blossom, date, rice, pepperco...",Ethiopia Shakiso Mormora-Revel Coffee,"[992, 1576, 1148, 1031, 2148]","[Estelí Nicaragua, Gigesa Ethiopia, Ethiopia N..."


In [74]:
import pickle
with open("data.p","wb") as f:
    pickle.dump(df,f)

In [75]:
data_dict = df.to_dict("index")
import pickle
with open("data.p","wb") as f:
    pickle.dump(data_dict,f)

In [76]:
d = {value["name"]:key for key,value in data_dict.items()}

In [77]:
with open("keys.p","wb") as f:
    pickle.dump(d,f)

In [78]:
with open("keys.p","wb") as f:
    pickle.dump(d,f)

In [79]:
df.to_csv("df.csv",index=False)

In [80]:
new = pd.read_csv("df.csv")
new.head(3)

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,description,desc_2,desc_3,descriptions,relevant_text,latitude,characteristics,coffee,recommendations,string_recommendation
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,...,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"Evaluated as espresso. Sweet-toned, deeply ric...","A.R.C.\n“Sweety” Espresso Blend\nHong Kong, Ch...",9.145000,['espresso' 'sweet' 'rich' 'chocolati' 'vanill...,“Sweety” Espresso Blend-A.R.C.,[1127 635 657 698 205],"['Kenya AA Ares Phoenix Special', 'D31 The New..."
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,...,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","Evaluated as espresso. Sweetly tart, floral-to...","A.R.C.\nFlora Blend Espresso\nHong Kong, China...",-8.783195,['espresso' 'floral' 'honeysuckl' 'oak' 'choco...,Flora Blend Espresso-A.R.C.,[1164 809 127 2253 710],"['Vertical Support Espresso', 'Zoom Espresso',..."
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,...,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","Crisply sweet, cocoa-toned. Lemon blossom, roa...",Revel Coffee\nEthiopia Shakiso Mormora\nBillin...,6.515691,['crispli' 'cocoa' 'blossom' 'date' 'rice' 'pe...,Ethiopia Shakiso Mormora-Revel Coffee,[ 992 1576 1148 1031 2148],"['Estelí Nicaragua', 'Gigesa Ethiopia', 'Ethio..."


In [81]:
def get_words(description):
    l = set([])
    try:
        words = nltk.word_tokenize(description.lower().replace("-"," "))
        #print(words)
        tagged = nltk.pos_tag(words)
        for i in tagged:
            if i[0] not in [",",".","!","?",":",";"]:
                l.add(stemmer.stem(i[0]))
        #print(tagged)


    except Exception as e:
        print(str(e))

    return l


In [82]:
words = set([])
for i in df["all_text"]:
    words.update(get_words(i))
words

{'duo',
 'coronado',
 '21.50/16',
 'packag',
 '815',
 'identifi',
 'jackfruit',
 'principito',
 'rubass',
 'luan',
 'scfcu',
 'gomez',
 'duromina',
 '5/12',
 'jam',
 'enriqu',
 'gunung',
 'mouthfeel',
 'label',
 '30.00/8',
 'fat',
 'remark',
 'vald',
 '6228',
 'fundación',
 'aguirr',
 'harrisonburg',
 'ojo',
 'murutra',
 'shake',
 'mai',
 'taipei',
 'cad',
 'hydro',
 'muslot',
 'rainha',
 '350/240',
 'huella',
 '1,300/8',
 'ozolotepec',
 'kakalov',
 'nyruzisa',
 'granja',
 'broad',
 'look',
 'tend',
 'symphon',
 'rhododendron',
 'ingin',
 'muhanga',
 'thung',
 'rought',
 'fumag',
 'rica/sumatra',
 'aguila',
 'www.bridgecoffeeco.com',
 '7018',
 'tannic',
 '1200/200',
 '34.95/6',
 'leaf',
 'seeth',
 '892',
 'kabunyeria',
 'manuel',
 'peacock',
 '18.50/16',
 'kukui',
 '15.99/16',
 'www.steadfast.coffe',
 'd.c.',
 '300/8',
 'rivet',
 'african',
 'dabay',
 'cachaça',
 'u.s.',
 'capitan',
 'been',
 'compot',
 'c26',
 '49.95/4',
 'kusillo',
 'esperanza',
 'kudum',
 'bonney',
 'www.m.facebook.

In [83]:
words = list(words)
with open("words.p","wb") as f:
    pickle.dump(words,f)

In [84]:
def one_hot(description):
    features = get_words(description)
    vector = np.array([1 if i in features else 0 for i in words])
    return vector

In [85]:
#vectors = [one_hot(i) for i in list(df["all_text"])]
df["all_words"] = df["all_text"].apply(one_hot)

In [86]:
df.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,desc_2,desc_3,descriptions,relevant_text,latitude,characteristics,coffee,recommendations,string_recommendation,all_words
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,...,An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"Evaluated as espresso. Sweet-toned, deeply ric...","A.R.C.\n“Sweety” Espresso Blend\nHong Kong, Ch...",9.145000,"[espresso, sweet, rich, chocolati, vanilla, ch...",“Sweety” Espresso Blend-A.R.C.,"[1127, 635, 657, 698, 205]","[Kenya AA Ares Phoenix Special, D31 The New Li...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,...,An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","Evaluated as espresso. Sweetly tart, floral-to...","A.R.C.\nFlora Blend Espresso\nHong Kong, China...",-8.783195,"[espresso, floral, honeysuckl, oak, chocol, th...",Flora Blend Espresso-A.R.C.,"[1164, 809, 127, 2253, 710]","[Vertical Support Espresso, Zoom Espresso, Sum...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,...,This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","Crisply sweet, cocoa-toned. Lemon blossom, roa...",Revel Coffee\nEthiopia Shakiso Mormora\nBillin...,6.515691,"[crispli, cocoa, blossom, date, rice, pepperco...",Ethiopia Shakiso Mormora-Revel Coffee,"[992, 1576, 1148, 1031, 2148]","[Estelí Nicaragua, Gigesa Ethiopia, Ethiopia N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRoast House\nEthiopia Suke Quto...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,...,This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,"Delicate, sweetly spice-toned. Pink peppercorn...","Roast House\nEthiopia Suke Quto\nSpokane, Wash...",9.066516,"[delic, peppercorn, date, myrrh, lavend, aroma...",Ethiopia Suke Quto-Roast House,"[1764, 1821, 1509, 104, 2138]","[Finca Huixoc Maracaturra, Skyway Fair Trade O...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,https://www.coffeereview.com/review/ethiopia-g...,\n\n\n94\n\n\nBig Creek Coffee Roasters\nEthi...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,...,Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,"Deeply sweet, subtly pungent. Honey, pear, tan...",Big Creek Coffee Roasters\nEthiopia Gedeb Halo...,5.917024,"[sweet, pungent, honey, pear, tangerin, zest, ...",Ethiopia Gedeb Halo Beriti-Big Creek Coffee Ro...,"[253, 31, 129, 504, 669]","[Ethiopia Deri Kochoha, Kenya Mutungati, Sumat...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [87]:
with open("df.p","wb") as f:
    pickle.dump(df,f)